# ROC With Simple Analysis

In [ ]:
import pathlib
import statistics
from typing import Literal

import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, recall_score, f1_score

from scripts.load_working_set import load_working_set

print("Done")

Some constants.

In [ ]:
# The location of the data
DATA_DIR = pathlib.Path("./data")
if not DATA_DIR.exists():
    raise FileNotFoundError(f"Data directory not found: '{DATA_DIR}'")

# The type of prediction to use
# See the cell with `def predict(...)` for the options
PREDICTION = "median"

Each sentence has a list of percentages associated with it.  
These need to be collapsed into a single percentage for now.  
I will be putting this functionality into a function, so I do not need to hunt for it later on.

In [ ]:
_predict_types = Literal["median", "mean", "harmonic_mean"]


def predict(l: list[float], _type: _predict_types) -> float:
    if _type == "median":
        return statistics.median(l)
    elif _type == "mean":
        return statistics.mean(l)
    elif _type == "harmonic_mean":
        return statistics.harmonic_mean(l)

    raise ValueError(f"Invalid predict type: {_type}")


print("Done")

Preparing our data from the working set.  
We create two arrays, one for the predictions and one for reality.  
We are using the fine-tuning subset of the working set to acheive this.

In [ ]:
# Load our data
sentence_evaluations, sentence_realities = load_working_set("fine-tuning")

# Convert the evaluations into single percentages via either mean or median
sentence_predictions: list[float] = []
for evaluations in sentence_evaluations:
    sentence_predictions.append(predict(evaluations, PREDICTION))

print("Done")

Now onto generating the ROC curve.

In [ ]:
fpr, tpr, thresholds = roc_curve(sentence_realities, sentence_predictions)
roc_auc = auc(fpr, tpr)

print("Done")

Now we plot the ROC curve.
With the best thresholds based of Youden's J statistic.

In [ ]:
# Maximising Youden's J Statistic
youden_index = tpr - fpr
best_threshold_index = np.argmax(youden_index)
best_threshold = thresholds[best_threshold_index]

# And plot again
plt.figure()
plt.plot(fpr, tpr, color="darkorange", lw=2, label=f"ROC curve (AUROC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")

# Plot the point of the best threshold
plt.scatter(fpr[best_threshold_index], tpr[best_threshold_index],
            color="red", label=f"Best Threshold (J={best_threshold:.2f})")
# And draw a vertical line underneath
plt.plot([fpr[best_threshold_index], fpr[best_threshold_index]],
         [tpr[best_threshold_index], fpr[best_threshold_index]],
         "r--")

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")

# Title
title = "Receiver Operating Characteristic (ROC)"
plt.title(title)

# Save
file = DATA_DIR / (title.replace(" ", "_") + ".png")
plt.savefig(file)

# Show
plt.show()

print(f"Best threshold = {best_threshold:.2f}")

print("Done")

Now, using the threshold calculated above to classify the evaluation set.

In [ ]:
sentence_evaluations_eval, sentence_realities_eval = load_working_set("evaluation")

sentence_predictions_eval: list[float] = []
for evaluations in sentence_evaluations_eval:
    sentence_predictions_eval.append(predict(evaluations, PREDICTION))

# Apply the threshold
classified_sentences_eval = [
    1 if p >= best_threshold else 0 for p in sentence_predictions_eval
]

# And calculate some performance metrics
accuracy = accuracy_score(sentence_realities_eval, classified_sentences_eval)
precision = precision_score(sentence_realities_eval, classified_sentences_eval)
recall = recall_score(sentence_realities_eval, classified_sentences_eval)
f1 = f1_score(sentence_realities_eval, classified_sentences_eval)

# Tells us the secrets
print(f"For threshold {best_threshold:.2f}\n")
print(f"\tAccuracy:  {accuracy:.2f}")
print(f"\tPrecision: {precision:.2f}")
print(f"\tRecall:    {recall:.2f}")
print(f"\tF1 Score:  {f1:.2f}")

# Create a simple file to store the data
data_dir = pathlib.Path("./data")
if not data_dir.exists():
    # This should never happen if the code reaches here
    # Well, unless you delete the dir yourself, please don't
    raise FileNotFoundError(f"Could not find data directory at {data_dir}")

results_file = data_dir / "roc_simple_results.txt"
with results_file.open("w") as f:
    print(f"Using analysis function '{PREDICTION}'", file=f)
    print(f"Best threshold {best_threshold:.2f}\n", file=f)
    print(f"\tAccuracy:  {accuracy:.2f}", file=f)
    print(f"\tPrecision: {precision:.2f}", file=f)
    print(f"\tRecall:    {recall:.2f}", file=f)
    print(f"\tF1 Score:  {f1:.2f}", file=f)

print()
print("Done")